# Chapter 8 Toolkit — Pattern Recognition + VC Theory (Lecture Notes pp. 123–145)

This notebook turns the chapter into a **reusable Python toolkit** 🧰

You’ll get:
- **Linear classifiers** (perceptron) ✅
- **Kernelization** (kernel perceptron + kernel checks) ✅
- **Empirical risk minimization (ERM)** + train/test guarantees ✅
- **Precision/Recall utilities** (with edge-case warnings) ✅
- **Finite-hyperplane ERM idea** (Thm 8.16) as a practical sampler ✅
- **VC theory bounds** (shattering upper bounds via Sauer–Shelah, generalization bounds) ✅
- **Martingale/Azuma-style concentration** for ERM risk (Thm 8.43) ✅

> Tip: Run top-to-bottom once. Then copy the “Function index” section whenever you solve exam-style problems.


In [ ]:
import numpy as np
import math
from dataclasses import dataclass
from typing import Callable, Dict, Tuple, Optional, List

import matplotlib.pyplot as plt


## 8.0 Setup: 0–1 loss, empirical risk, and metrics 🎯

The chapter’s core object is the **risk**:
- True risk:  \(R(g)=\mathbb{E}[\mathbf{1}\{Y\neq g(X)\}]\)
- Empirical risk:  \(\hat R_n(g)=\frac1n\sum_{i=1}^n \mathbf{1}\{Y_i\neq g(X_i)\}\)

Below are utilities you can reuse for any classifier.


In [ ]:
def to_pm1(y: np.ndarray) -> np.ndarray:
    """Convert labels in {0,1} to {-1,+1}. If already {-1,+1}, keep."""
    y = np.asarray(y).astype(int)
    vals = set(np.unique(y).tolist())
    if vals.issubset({-1, 1}):
        return y
    if vals.issubset({0, 1}):
        return 2*y - 1
    raise ValueError("y must be in {0,1} or {-1,1}.")

def to_01(y_pm1: np.ndarray) -> np.ndarray:
    """Convert labels in {-1,+1} to {0,1}."""
    y = np.asarray(y_pm1).astype(int)
    vals = set(np.unique(y).tolist())
    if not vals.issubset({-1, 1}):
        raise ValueError("y must be in {-1,1}.")
    return (y + 1)//2

def zero_one_loss(y_true: np.ndarray, y_pred: np.ndarray) -> np.ndarray:
    y_true = np.asarray(y_true).astype(int)
    y_pred = np.asarray(y_pred).astype(int)
    return (y_true != y_pred).astype(float)

def empirical_risk(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    return float(np.mean(zero_one_loss(y_true, y_pred)))

@dataclass
class Confusion:
    tp: int; fp: int; tn: int; fn: int

def confusion_matrix_01(y_true01: np.ndarray, y_pred01: np.ndarray) -> Confusion:
    y_true01 = np.asarray(y_true01).astype(int)
    y_pred01 = np.asarray(y_pred01).astype(int)
    tp = int(np.sum((y_true01==1) & (y_pred01==1)))
    fp = int(np.sum((y_true01==0) & (y_pred01==1)))
    tn = int(np.sum((y_true01==0) & (y_pred01==0)))
    fn = int(np.sum((y_true01==1) & (y_pred01==0)))
    return Confusion(tp=tp, fp=fp, tn=tn, fn=fn)

def precision_recall_f1(y_true01: np.ndarray, y_pred01: np.ndarray) -> Dict[str, float]:
    C = confusion_matrix_01(y_true01, y_pred01)
    precision = C.tp / (C.tp + C.fp) if (C.tp + C.fp) > 0 else float("nan")
    recall = C.tp / (C.tp + C.fn) if (C.tp + C.fn) > 0 else float("nan")
    f1 = 2*precision*recall/(precision+recall) if (precision==precision and recall==recall and (precision+recall)>0) else float("nan")
    acc = (C.tp + C.tn) / max(1, (C.tp + C.fp + C.tn + C.fn))
    return {"accuracy": float(acc), "precision": float(precision), "recall": float(recall), "f1": float(f1), **C.__dict__}


### Held-out test set guarantee (Hoeffding style) ✅

If you train on `n` points and test on an **independent** test set of size `m`, then for bounded loss (like 0–1 loss):
\[
\mathbb{P}\big(|\hat R_m(\hat g)-R(\hat g)|>\varepsilon\big)\le 2e^{-2m\varepsilon^2}
\]
This matches the chapter’s idea: **test risk concentrates** because the test set is independent of the fitted model.


In [ ]:
def hoehffding_test_bound(m: int, eps: float) -> float:
    """2 * exp(-2 m eps^2) for bounded loss in [0,1]."""
    if m <= 0 or eps <= 0:
        raise ValueError("m>0 and eps>0 required.")
    return float(2 * math.exp(-2 * m * eps * eps))

def required_test_size_for_eps_delta(eps: float, delta: float) -> int:
    """Small helper: choose m so that 2 exp(-2 m eps^2) <= delta."""
    if not (0 < delta < 1) or eps <= 0:
        raise ValueError("delta in (0,1), eps>0.")
    m = math.log(2/delta) / (2 * eps * eps)
    return int(math.ceil(m))

# Example:
eps, delta = 0.03, 0.05
print("m needed for eps=0.03, delta=0.05:", required_test_size_for_eps_delta(eps, delta))
print("bound check:", hoehffding_test_bound(required_test_size_for_eps_delta(eps, delta), eps))


## 8.1 Linear classifiers — Perceptron 🧠

A linear separator predicts with:
- labels in {-1,+1}:  \(\hat y = \mathrm{sign}(w^\top x + b)\)

Perceptron update (as in notes, with augmented coordinate trick):
- if \(y_i (w^\top x_i + b)\le 0\), update \(w \leftarrow w + y_i x_i\), \(b \leftarrow b + y_i\)

Below is a clean reusable implementation.


In [ ]:
@dataclass
class PerceptronModel:
    w: np.ndarray
    b: float

def perceptron_train(
    X: np.ndarray,
    y_pm1: np.ndarray,
    max_epochs: int = 50,
    shuffle: bool = True,
    seed: int = 0,
) -> Tuple[PerceptronModel, Dict[str, object]]:
    """Train a perceptron. y must be in {-1,+1}."""
    X = np.asarray(X, dtype=float)
    y = to_pm1(y_pm1).astype(int)
    n, d = X.shape
    rng = np.random.default_rng(seed)

    w = np.zeros(d, dtype=float)
    b = 0.0
    updates = 0

    for epoch in range(max_epochs):
        idx = np.arange(n)
        if shuffle:
            rng.shuffle(idx)
        mistakes = 0
        for i in idx:
            margin = y[i] * (X[i] @ w + b)
            if margin <= 0:
                w = w + y[i] * X[i]
                b = b + y[i]
                updates += 1
                mistakes += 1
        if mistakes == 0:
            return PerceptronModel(w=w, b=b), {"converged": True, "epochs": epoch+1, "updates": updates}
    return PerceptronModel(w=w, b=b), {"converged": False, "epochs": max_epochs, "updates": updates}

def perceptron_predict(model: PerceptronModel, X: np.ndarray) -> np.ndarray:
    X = np.asarray(X, dtype=float)
    scores = X @ model.w + model.b
    yhat = np.where(scores >= 0, 1, -1).astype(int)
    return yhat


### Demo: linearly separable vs non-separable

- If separable, perceptron should converge (Theorem 8.2).
- If not separable, it may cycle → use a fixed epoch budget.


In [ ]:
def make_linearly_separable(n: int = 200, seed: int = 0) -> Tuple[np.ndarray, np.ndarray]:
    rng = np.random.default_rng(seed)
    X = rng.normal(size=(n, 2))
    true_w = np.array([1.2, -0.7])
    true_b = 0.1
    y = np.where(X @ true_w + true_b >= 0, 1, -1)
    return X, y

def make_ring_nonseparable(n: int = 300, seed: int = 0) -> Tuple[np.ndarray, np.ndarray]:
    rng = np.random.default_rng(seed)
    angles = rng.uniform(0, 2*np.pi, size=n)
    radii = rng.uniform(0.0, 4.0, size=n)
    X = np.column_stack([radii*np.cos(angles), radii*np.sin(angles)])
    y = np.where(radii <= 1.0, 1, -1)  # inner disk vs outer region (not linearly separable)
    return X, y

def plot_2d_points(X: np.ndarray, y_pm1: np.ndarray, title: str = ""):
    y = to_pm1(y_pm1)
    plt.figure()
    plt.scatter(X[y==1, 0], X[y==1, 1], s=18, label="+1")
    plt.scatter(X[y==-1, 0], X[y==-1, 1], s=18, label="-1")
    plt.title(title)
    plt.xlabel("x1")
    plt.ylabel("x2")
    plt.legend()
    plt.show()

X_sep, y_sep = make_linearly_separable()
plot_2d_points(X_sep, y_sep, "Linearly separable data")

model, info = perceptron_train(X_sep, y_sep, max_epochs=30, seed=0)
yhat = perceptron_predict(model, X_sep)
print(info, "train risk:", empirical_risk(y_sep, yhat))

X_ring, y_ring = make_ring_nonseparable()
plot_2d_points(X_ring, y_ring, "Non-separable (ring) data")

model2, info2 = perceptron_train(X_ring, y_ring, max_epochs=30, seed=0)
yhat2 = perceptron_predict(model2, X_ring)
print(info2, "train risk:", empirical_risk(y_ring, yhat2))


## 8.2 Kernelization 🔧

If data isn’t linearly separable in input space, map features via \(\phi(x)\) so it becomes separable in feature space.

Key trick in the notes:
- Perceptron weights become a combination of training points: \(w=\sum_i c_i \phi(x_i)\)
- Predictions use only **kernel evaluations** \(k(x_i,x)=\phi(x_i)\cdot \phi(x)\)

This gives the **Kernel Perceptron**.


In [ ]:
# -----------------------------
# Common kernels (use whichever you need)
# -----------------------------

def kernel_linear(X: np.ndarray, Y: np.ndarray) -> np.ndarray:
    return np.asarray(X) @ np.asarray(Y).T

def kernel_poly(X: np.ndarray, Y: np.ndarray, gamma: float = 1.0, r: float = 0.0, degree: int = 2) -> np.ndarray:
    return (gamma * (np.asarray(X) @ np.asarray(Y).T) + r) ** degree

def kernel_rbf_l2(X: np.ndarray, Y: np.ndarray, gamma: float = 1.0) -> np.ndarray:
    X = np.asarray(X); Y = np.asarray(Y)
    X2 = np.sum(X*X, axis=1, keepdims=True)
    Y2 = np.sum(Y*Y, axis=1, keepdims=True).T
    d2 = X2 + Y2 - 2*(X @ Y.T)
    return np.exp(-gamma * d2)

def kernel_rbf_l1(X: np.ndarray, Y: np.ndarray, gamma: float = 1.0) -> np.ndarray:
    X = np.asarray(X); Y = np.asarray(Y)
    K = np.empty((X.shape[0], Y.shape[0]), dtype=float)
    for i in range(X.shape[0]):
        K[i, :] = np.exp(-gamma * np.sum(np.abs(Y - X[i]), axis=1))
    return K

def is_psd(K: np.ndarray, tol: float = 1e-10) -> bool:
    """Check symmetric PSD: eigenvalues >= -tol."""
    K = np.asarray(K, dtype=float)
    if not np.allclose(K, K.T, atol=1e-8):
        return False
    eigs = np.linalg.eigvalsh(K)
    return bool(np.min(eigs) >= -tol)

def feature_map_from_gram(K: np.ndarray, tol: float = 1e-12) -> np.ndarray:
    """Lemma 8.3 constructive: if K is PSD, build B s.t. K = B B^T, rows are phi(x_i)."""
    K = np.asarray(K, dtype=float)
    if not np.allclose(K, K.T, atol=1e-8):
        raise ValueError("K must be symmetric.")
    eigvals, eigvecs = np.linalg.eigh(K)
    eigvals = np.maximum(eigvals, 0.0)
    keep = eigvals > tol
    B = eigvecs[:, keep] * np.sqrt(eigvals[keep])
    return B


In [ ]:
@dataclass
class KernelPerceptronModel:
    X_train: np.ndarray
    y_train: np.ndarray   # {-1,+1}
    alpha: np.ndarray     # integer-like update counts
    kernel: Callable[[np.ndarray, np.ndarray], np.ndarray]

def kernel_perceptron_train(
    X: np.ndarray,
    y_pm1: np.ndarray,
    kernel: Callable[[np.ndarray, np.ndarray], np.ndarray],
    max_epochs: int = 20,
    shuffle: bool = True,
    seed: int = 0,
) -> Tuple[KernelPerceptronModel, Dict[str, object]]:
    X = np.asarray(X, dtype=float)
    y = to_pm1(y_pm1).astype(int)
    n = X.shape[0]
    rng = np.random.default_rng(seed)
    alpha = np.zeros(n, dtype=float)

    K = kernel(X, X)
    updates = 0

    for epoch in range(max_epochs):
        idx = np.arange(n)
        if shuffle:
            rng.shuffle(idx)
        mistakes = 0
        for i in idx:
            # f(x_i) = sum_j alpha_j y_j k(x_j, x_i)
            score = np.sum(alpha * y * K[:, i])
            if y[i] * score <= 0:
                alpha[i] += 1.0
                updates += 1
                mistakes += 1
        if mistakes == 0:
            return KernelPerceptronModel(X, y, alpha, kernel), {"converged": True, "epochs": epoch+1, "updates": updates}
    return KernelPerceptronModel(X, y, alpha, kernel), {"converged": False, "epochs": max_epochs, "updates": updates}

def kernel_perceptron_predict(model: KernelPerceptronModel, X_new: np.ndarray) -> np.ndarray:
    X_new = np.asarray(X_new, dtype=float)
    K = model.kernel(model.X_train, X_new)  # shape (n_train, n_new)
    scores = (model.alpha * model.y_train)[:, None] * K
    scores = np.sum(scores, axis=0)
    return np.where(scores >= 0, 1, -1).astype(int)


### Demo: kernel fixes the ring example

Try:
- Linear kernel (should struggle)
- Polynomial kernel or RBF kernel (should work much better)


In [ ]:
# Linear kernel perceptron on ring
kp_lin, info_lin = kernel_perceptron_train(X_ring, y_ring, kernel=lambda A,B: kernel_linear(A,B), max_epochs=15, seed=0)
pred_lin = kernel_perceptron_predict(kp_lin, X_ring)
print("linear-kernel:", info_lin, "train risk:", empirical_risk(y_ring, pred_lin))

# RBF kernel perceptron on ring
kp_rbf, info_rbf = kernel_perceptron_train(X_ring, y_ring, kernel=lambda A,B: kernel_rbf_l2(A,B,gamma=1.0), max_epochs=15, seed=0)
pred_rbf = kernel_perceptron_predict(kp_rbf, X_ring)
print("rbf-kernel:", info_rbf, "train risk:", empirical_risk(y_ring, pred_rbf))


## 8.3 Empirical Risk Minimization (ERM) + test-set guarantee ✅

Given a model class \(M\), ERM chooses:
\[
\hat g_n \in \arg\min_{g\in M}\hat R_n(g)
\]

Important: \(\hat R_n(\hat g_n)\) is typically **downward biased**, because you optimized it on the same data.

If you have an independent test set, you can bound the gap between test risk and true risk using Hoeffding (earlier cell).


In [ ]:
def train_test_split(X: np.ndarray, y: np.ndarray, test_size: float = 0.3, seed: int = 0):
    X = np.asarray(X); y = np.asarray(y)
    n = X.shape[0]
    rng = np.random.default_rng(seed)
    idx = np.arange(n); rng.shuffle(idx)
    m = int(round(n * test_size))
    test_idx = idx[:m]; train_idx = idx[m:]
    return X[train_idx], X[test_idx], y[train_idx], y[test_idx]

# Example: evaluate perceptron with a test set + Hoeffding bound
Xtr, Xte, ytr, yte = train_test_split(X_sep, y_sep, test_size=0.3, seed=1)
model, info = perceptron_train(Xtr, ytr, max_epochs=30, seed=0)
pred_te = perceptron_predict(model, Xte)
risk_te = empirical_risk(yte, pred_te)

eps = 0.05
print("test risk:", risk_te)
print("P(|Rhat - R| > eps) <=", hoehffding_test_bound(len(yte), eps))


## 8.3.2 Precision & Recall (and why edge cases are tricky) ⚠️

Definitions for class 1:
- Precision = P(Y=1 | g(X)=1)
- Recall    = P(g(X)=1 | Y=1)  (also called sensitivity)

On a finite test set:
- Precision uses the denominator **#predicted positive**
- Recall uses the denominator **#actual positive**

So if your model predicts all 0, precision is undefined; if positives are rare, recall estimation is noisy.
Utilities below compute metrics safely and help you spot these cases.


In [ ]:
def metrics_on_testset(y_true, y_pred, label_space: str = "pm1") -> Dict[str, float]:
    """label_space: 'pm1' for {-1,1}, '01' for {0,1}."""
    if label_space == "pm1":
        y_true01 = to_01(to_pm1(y_true))
        y_pred01 = to_01(to_pm1(y_pred))
    elif label_space == "01":
        y_true01 = np.asarray(y_true).astype(int)
        y_pred01 = np.asarray(y_pred).astype(int)
    else:
        raise ValueError("label_space must be 'pm1' or '01'")
    return precision_recall_f1(y_true01, y_pred01)

# Edge-case demos
y_true01 = np.array([1,0,0,0,0,0,0,0,0,0])
always1 = np.ones_like(y_true01)
always0 = np.zeros_like(y_true01)
print("always 1:", metrics_on_testset(y_true01, always1, label_space="01"))
print("always 0:", metrics_on_testset(y_true01, always0, label_space="01"))


## 8.4 ERM over finitely many hyperplanes (Thm 8.16) — practical sampler 🎲

The notes describe a finite class \(M_n\) built from **hyperplanes defined by choosing m points** (feature dimension \(m\)).
There are \(2\binom{n}{m}\) such rules (two sides of each hyperplane).

In practice, \(\binom{n}{m}\) is huge. For exams and coding, a good approach is:
- **sample** many random m-tuples
- build hyperplanes
- pick the one with smallest training error

Below is a reusable implementation for that idea.


In [ ]:
def hyperplane_from_d_points(points: np.ndarray, tol: float = 1e-12) -> Tuple[np.ndarray, float]:
    """
    Build a hyperplane a^T x + b = 0 passing through d points in R^d.
    Returns (a, b) up to scaling.
    Uses nullspace of [points, 1].
    """
    P = np.asarray(points, dtype=float)
    d = P.shape[1]
    if P.shape[0] != d:
        raise ValueError("Need exactly d points in R^d to determine a hyperplane uniquely (up to scale).")
    A = np.hstack([P, np.ones((d, 1))])  # shape (d, d+1)
    # Find vector v in nullspace of A (A v = 0). Nullspace dimension should be 1 in general position.
    U, S, Vt = np.linalg.svd(A)
    v = Vt[-1, :]  # smallest singular value vector
    if np.linalg.norm(A @ v) > 1e-8:
        raise RuntimeError("Failed to find a stable nullspace vector; points may be degenerate.")
    a = v[:d]
    b = v[d]
    # normalize for stability
    norm = np.linalg.norm(a)
    if norm < tol:
        raise RuntimeError("Degenerate hyperplane (normal ~ 0).")
    a = a / norm
    b = b / norm
    return a, b

def predict_hyperplane(a: np.ndarray, b: float, X: np.ndarray) -> np.ndarray:
    scores = np.asarray(X, dtype=float) @ a + b
    return np.where(scores >= 0, 1, 0).astype(int)  # {0,1} decision rule

def sampled_hyperplane_erm(
    X: np.ndarray,
    y01: np.ndarray,
    n_candidates: int = 2000,
    seed: int = 0,
) -> Dict[str, object]:
    """Sample many d-tuples, build hyperplanes, pick best empirical-risk rule."""
    X = np.asarray(X, dtype=float)
    y01 = np.asarray(y01).astype(int)
    n, d = X.shape
    if n < d:
        raise ValueError("Need n >= d to sample d points.")
    rng = np.random.default_rng(seed)

    best = {"risk": float("inf"), "a": None, "b": None, "flip": 0}
    for _ in range(n_candidates):
        idx = rng.choice(n, size=d, replace=False)
        try:
            a, b = hyperplane_from_d_points(X[idx, :])
        except Exception:
            continue
        pred = predict_hyperplane(a, b, X)
        r = empirical_risk(y01, pred)
        # also consider the opposite side (phi+ vs phi-)
        pred_flip = 1 - pred
        r2 = empirical_risk(y01, pred_flip)
        if r2 < r:
            r, pred = r2, pred_flip
            flip = 1
        else:
            flip = 0
        if r < best["risk"]:
            best = {"risk": float(r), "a": a, "b": float(b), "flip": flip}
    return best


In [ ]:
# Demo in 2D (d=2): sample hyperplane ERM on separable data (convert labels to {0,1})
best = sampled_hyperplane_erm(X_sep, to_01(y_sep), n_candidates=3000, seed=0)
best["risk"], best["a"], best["b"], best["flip"]


### Theorem 8.16 bound (as a function) 📏

The notes give a probability bound of the form:
\[
\mathbb{P}\{R(\hat\phi) > \inf_{M} R(\phi) + \varepsilon\}
\le \exp(2m\varepsilon)\,\big(2\binom{n}{m}+1\big)\,\exp(-n\varepsilon^2/2)
\]
(valid under conditions like \(2m/n \le \varepsilon \le 1\)).

Below: compute it numerically.


In [ ]:
def comb(n: int, k: int) -> int:
    return math.comb(n, k)

def theorem_816_bound(n: int, m_dim: int, eps: float) -> float:
    if not (0 < eps <= 1):
        raise ValueError("eps in (0,1]")
    # exp(2 m eps) * (2*C(n,m)+1) * exp(-n eps^2 / 2)
    return float(math.exp(2*m_dim*eps) * (2*comb(n, m_dim) + 1) * math.exp(-n*(eps**2)/2))

# Example numbers (small just for illustration)
print(theorem_816_bound(n=200, m_dim=2, eps=0.2))


## 8.5–8.7 VC theory: shattering number and generalization bounds 📚

The notes re-express uniform convergence as:
\[
\mathbb{P}\Big(\sup_{A\in\mathcal{A}}|\nu_n(A)-\nu(A)|>\varepsilon\Big)
\]

Key objects:
- **Shattering number** \(s(\mathcal{A},n)\)
- **VC dimension** \(V_{\mathcal{A}}\)
- **Sauer–Shelah**: \(s(\mathcal{A},n)\le \sum_{i=0}^{V-1}\binom{n}{i}\)

Theorem 8.29 gives:
\[
\mathbb{P}\Big(\sup_{A\in\mathcal{A}}|\nu_n(A)-\nu(A)|>\varepsilon\Big)
\le 8\,s(\mathcal{A},n)\,e^{-n\varepsilon^2/32}
\]
(under \(n\varepsilon^2\ge 2\)).

Corollary 8.31 adjusts constants for classifiers (under \(n\varepsilon^2\ge 8\)).


In [ ]:
def sauer_shelah_upper(n: int, vc_dim: int) -> int:
    """Upper bound on shattering number via Sauer–Shelah: sum_{i=0}^{V-1} C(n,i)."""
    if vc_dim <= 0:
        return 1
    upper = 0
    for i in range(0, min(vc_dim, n+1)):
        upper += math.comb(n, i)
    return upper

def lemma_837_poly_upper(N: int, k: int) -> float:
    """Lemma 8.37 style: sum_{i=0}^{k-1} C(N,i) <= (eN/k)^k."""
    if k <= 0:
        return 1.0
    return float((math.e * N / k) ** k)

def vc_generalization_bound_theorem_829(n: int, eps: float, shattering_number: float) -> float:
    if eps <= 0:
        raise ValueError("eps>0")
    return float(8.0 * shattering_number * math.exp(-n * eps * eps / 32.0))

def vc_generalization_bound_cor_831(n: int, eps: float, shattering_number: float) -> float:
    if eps <= 0:
        raise ValueError("eps>0")
    return float(8.0 * shattering_number * math.exp(-n * eps * eps / 64.0))

# Example: linear classifiers in R^m have VC-dim about m+1 (Lemma 8.38 informal)
def bound_linear_classifiers(n: int, eps: float, m_dim: int) -> Dict[str, float]:
    vc_dim = m_dim + 1
    s_upper = sauer_shelah_upper(n, vc_dim)
    return {
        "vc_dim≈m+1": vc_dim,
        "sauer_shelah_upper": float(s_upper),
        "Thm8.29_bound": vc_generalization_bound_theorem_829(n, eps, s_upper),
        "Cor8.31_bound": vc_generalization_bound_cor_831(n, eps, s_upper),
    }

bound_linear_classifiers(n=200, eps=0.1, m_dim=2)


## 8.8–8.9 Martingale concentration for ERM empirical risk (Thm 8.43) 🎲

The notes show:
\[
\mathbb{P}\Big(|\hat R_n(\phi_n^*) - \mathbb{E}[\hat R_n(\phi_n^*)]| > \varepsilon\Big)
< 2 e^{-n\varepsilon^2/2}
\]

This is an Azuma/Hoeffding-style concentration for the **random ERM training error** itself.

Below is a helper that computes that bound.


In [ ]:
def erm_training_risk_concentration_bound(n: int, eps: float) -> float:
    """Thm 8.43: 2 exp(-n eps^2 / 2)."""
    if n <= 0 or eps <= 0:
        raise ValueError("n>0, eps>0.")
    return float(2.0 * math.exp(-n * eps * eps / 2.0))

print(erm_training_risk_concentration_bound(500, 0.05))


## ✅ Function index (copy/paste friendly)

### Core risk + metrics
- `empirical_risk(y_true, y_pred)`
- `precision_recall_f1(y_true01, y_pred01)` / `metrics_on_testset(...)`
- `train_test_split(X,y, test_size, seed)`
- `hoehffding_test_bound(m, eps)` / `required_test_size_for_eps_delta(eps, delta)`

### Linear classifiers
- `perceptron_train(X, y_pm1, max_epochs, shuffle, seed)`
- `perceptron_predict(model, X)`

### Kernels + kernel perceptron
- `kernel_linear`, `kernel_poly`, `kernel_rbf_l2`, `kernel_rbf_l1`
- `is_psd(K)` and `feature_map_from_gram(K)` (Lemma 8.3 constructive)
- `kernel_perceptron_train(X, y_pm1, kernel, ...)`
- `kernel_perceptron_predict(model, X_new)`

### Finite hyperplane ERM idea (Thm 8.16)
- `hyperplane_from_d_points(points)`
- `sampled_hyperplane_erm(X, y01, n_candidates, seed)`
- `theorem_816_bound(n, m_dim, eps)`

### VC theory bounds
- `sauer_shelah_upper(n, vc_dim)`
- `vc_generalization_bound_theorem_829(n, eps, s)`
- `vc_generalization_bound_cor_831(n, eps, s)`
- `bound_linear_classifiers(n, eps, m_dim)`

### Martingale concentration (Thm 8.43)
- `erm_training_risk_concentration_bound(n, eps)`
